In [1]:
from tasks import DataFetchingTask, DataCalculationTask
from external.client import YandexWeatherAPI

client = YandexWeatherAPI()
data_fetcher = DataFetchingTask(client)
data_fetcher.run(max_workers=5)

Unable to fetch data for GIZA: HTTP Error 404: Not Found
Unable to fetch data for TORONTO: Extra data: line 19 column 4 (char 328)


In [2]:
available_cities = data_fetcher.city_to_forecasting_data.keys()
print('Available cities:', ', '.join(available_cities))

Available cities: LONDON, MOSCOW, BEIJING, PARIS, BERLIN, KAZAN, VOLGOGRAD, SPETERSBURG, KALININGRAD, NOVOSIBIRSK, ABUDHABI, BUCHAREST, ROMA, CAIRO, WARSZAWA, MADRID


In [3]:
DataCalculationTask(data_fetcher.city_to_forecasting_data

AttributeError: 'DataCalculationTask' object has no attribute 'city_to_forecasting_data'

{'date': '2022-05-26', 'hours_start': 9, 'hours_end': 19, 'hours_count': 11, 'temp_avg': 19.273, 'relevant_cond_hours': 9}
{'date': '2022-05-27', 'hours_start': 9, 'hours_end': 19, 'hours_count': 11, 'temp_avg': 16.0, 'relevant_cond_hours': 6}
{'date': '2022-05-28', 'hours_start': 9, 'hours_end': 19, 'hours_count': 11, 'temp_avg': 13.636, 'relevant_cond_hours': 0}
{'date': '2022-05-29', 'hours_start': None, 'hours_end': None, 'hours_count': 0, 'temp_avg': None, 'relevant_cond_hours': 0}
{'date': '2022-05-30', 'hours_start': None, 'hours_end': None, 'hours_count': 0, 'temp_avg': None, 'relevant_cond_hours': 0}
